Ball on a beam
====


Øvelsen har til hensigt at øve den studerende i den mange gange næsten umulige opgave, der ligger i at arbejde med et neuraltnetværk.

Hjemmelavet træningsdata med cirka 5000 data har givet lidt forskellige modeller. 

Jeg har fundet, at file1.keras og file500.keras virker bedst.

At træne et neuralt netværk med data generet af et neuralt netværk lader ikke til at virke særlig godt. 

Det mest opsigtsvækkende ved denne opgave er, at vi forsøger at løse den med et neuralt netværk.

$$
angle = \frac{position-center}{100}^2 speed
$$

In [1]:
import pygame, sys
from pygame.locals import *
import pygame_widgets
from pygame_widgets.slider import Slider

import random
import numpy as np
import time
import tensorflow as tf
import keras

pygame 2.6.1 (SDL 2.28.4, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Boab:
    def __init__(self, model_file):
        #global model,slider,acc,speed,angle,text,textRect,screen,red,white,green,blue,black,bg,Press,downwards,x,clock,sound1,sound2,sound3,b1,b2,es
        # pygame stuff
        pygame.init()
        self.screen = pygame.display.set_mode((1500, 500))
        self.slider = Slider(self.screen, 100, 30, 800, 20, min=0, max=99, step=1)
        pygame.display.set_caption("Ball on a Beam")
        #sound1 =  pygame.mixer.Sound("womp.wav")
        self.clock = pygame.time.Clock()
        self.done = False
        self.white=(255,255,255)
        self.red = (255,0,0)
        self.green = (0,255,0)
        self.blue = (0,0,255)
        self.black = (0,0,0)
        self.bg = (127,127,127)

        # ball and beam stuff
        self.speed = 1
        self.acc = 0
        self.x = 500
        self.angle = 0
        self.model = keras.models.load_model(model_file)
        self.hits = 0

    
    def run_model(self, model_active=False):
        #global model,slider,acc,speed,angle,text,textRect,screen, red, white, green,blue,black, bg, Press, downwards, x,clock,sound1,sound2,sound3,b1,b2,es
        # eller også bruger man det neurale netværk
        if model_active is True:
            self.angle = self.model.predict(np.array([(self.x,self.speed)]),verbose=0)
            self.angle = self.angle[0][0]
        else:
            #self.angle = (self.slider.getValue()-50)

            # no neural network needed with this function.
            if self.x > 500:
                self.angle = -((self.x-500)/100)**2*abs(self.speed)
            else:
                self.angle = ((self.x-500)/100)**2*abs(self.speed)
        
        ##Dynamiske ligninger som giver bolden accelleration hastighed og position
        #self.acc = self.angle/30
        self.acc = self.angle
        nois = 1 * (random.random()-0.5)
        self.speed += self.acc + nois # støj kan lægges til +
        self.x += self.speed
        
        #denne linje kan generere CSV-data på konsol.
        #kopier dem og læg dem ind i en CSV-fil
        if (self.x < 5):
            self.speed = -self.speed
            self.hits += 1
            print('Hit')
            #sound1.play()
        if (self.x > 1200):
            self.speed = -self.speed
            self.hits += 1
            print('Hit')
            #sound1.play()
    
        print(f"{self.x},{self.angle},{self.speed}\n")
        train_data.write(f"{self.x},{self.angle},{self.speed}\n")

        # pygame stuff 
        # draw elements in the screen
        self.screen.fill(self.bg)
        pygame.draw.rect(self.screen, self.red, pygame.Rect(100, 200, 1300, 100), 49)  # beam
        pygame.draw.circle(self.screen, self.green, [750,400],100)  #support
        pygame.draw.rect(self.screen, self.green, pygame.Rect(650, 400, 200, 100), 49)  # support
        pygame.draw.circle(self.screen, self.blue, [self.x+100,150],50)  #The Ball
        
        font = pygame.font.Font('freesansbold.ttf', 32)
        text = font.render('Beam Angle : '+str(self.angle)+' deg', True, self.green, self.blue)
        textRect = text.get_rect()
        textRect.center = (400, 400)
        self.screen.blit(text, textRect)
        
        events = pygame.event.get()
        pygame_widgets.update(events)  # hmmm
        pygame.display.update()
        self.clock.tick(20)   #20 fps
        #print(self.clock.get_fps())
        
        for event in events:
            pass
            if event.type == pygame.QUIT:
                self.done = True
                # deactivates the pygame library
                pygame.quit()
                #break
                # quit the program.
                #quit()
        

In [3]:
"""
boab_training_data = Boab('file.keras')

with open('training-data.csv', 'w') as train_data:
    train_data.write('Ball position,Angle,Speed\n')
    while True:
        boab_training_data.run_model()
"""

"\nboab_training_data = Boab('file.keras')\n\nwith open('training-data.csv', 'w') as train_data:\n    train_data.write('Ball position,Angle,Speed\n')\n    while True:\n        boab_training_data.run_model()\n"

In [4]:
#model = keras.models.load_model('file.keras')
boab_model = Boab('file500.keras')
#boab_model = Boab('file2.keras')

with open('training-data-model.csv', 'w') as train_data:
    train_data.write('Ball position,Angle,Speed\n')
    while True:
        boab_model.run_model(model_active=True)
        #time.sleep(.1)

508.8642915127307,7.715742588043213,8.864291512730716

524.9572331150217,7.510331630706787,16.09294160229106

548.5133604484583,7.053786277770996,23.556127333436585

578.0722343196143,6.0995025634765625,29.558873871155967

611.572087232024,4.334846019744873,33.49985291240983

646.2346244108077,1.6437745094299316,34.662537178783595

679.33358462463,-1.455647349357605,33.09896021382232

708.4705877010728,-4.179741382598877,29.13700307644271

731.7117738166061,-6.147250175476074,23.241186115533264

747.4393752301019,-7.395707607269287,15.727601413495766

755.0807774961185,-8.079205513000488,7.641402266016641

754.539698120829,-8.347029685974121,-0.5410793752894598

746.2058866798624,-8.266714096069336,-8.33381144096663

729.5505239389787,-7.8397698402404785,-16.65536274088368

705.7551287464461,-6.925863265991211,-23.795395192532617

676.8066865766161,-5.391815662384033,-28.948442169829903

644.4926400389892,-3.1491506099700928,-32.31404653762693

611.5278865099946,-0.2918582558631897,-32

error: display Surface quit

In [16]:
from random import random, uniform
import time

def create_training_data():
    ball_speed = 0
    ball_position = 0
    left_edge = -100
    right_edge = 100
    angle = 0
    noise = 0
    
    with open('training-data-v1.csv', 'w') as train_data:
        train_data.write('Ball position,Angle,Speed\n')
        
        while True:
            angle = (ball_position/100)*2*ball_speed
            print(angle)
            noise = random()-0.5
            ball_speed += angle + noise
            ball_position += ball_speed
            
            if (ball_position < left_edge):
                ball_speed = -ball_speed
                print('hit')
            if (ball_position > right_edge):
                ball_speed = -ball_speed
                print('hit')
            time.sleep(.1)
            print(f"{ball_position},{angle},{ball_speed},\n")
            train_data.write(f"{ball_position},{angle},{ball_speed},\n")

create_training_data()

0.0
-0.279972189288175,0.0,-0.279972189288175,

0.001567688535496274
-0.1680083667776091,0.001567688535496274,0.11196382251056591,

-0.0003762171791635657
-0.33518393827057646,-0.0003762171791635657,-0.16717557149296736,

0.0011206913287129423
-0.36747007021759515,0.0011206913287129423,-0.03228613194701868,

0.00023728374347250993
-0.3369343884580065,0.00023728374347250993,0.030535681759588623,

-0.00020577042519630593
-0.7342027854481048,-0.00020577042519630593,-0.3972683969900983,

0.005833511272812673
-1.4293264151620968,0.005833511272812673,-0.695123629713992,

0.0198711713150713
-1.8945716052642547,0.0198711713150713,-0.4652451901021579,

0.017628806533066376
-2.422825117585669,0.017628806533066376,-0.5282535123214145,

0.025597317562103476
-2.4390128907421604,0.025597317562103476,-0.016187773156491225,

0.0007896437480218401
-2.7202362589817377,0.0007896437480218401,-0.2812233682395773,

0.015299880063165426
-2.7062234421424756,0.015299880063165426,0.014012816839262032,

-0.00075

KeyboardInterrupt: 